## Въведение в регресията - Урок 1

#### Поставяне в перспектива

✅ Съществуват много видове методи за регресия, и изборът на подходящия зависи от въпроса, на който искате да отговорите. Ако искате да предвидите вероятната височина на човек на определена възраст, бихте използвали `линейна регресия`, тъй като търсите **числова стойност**. Ако се интересувате дали даден тип кухня трябва да се счита за веганска или не, търсите **категорийно определение**, затова бихте използвали `логистична регресия`. Ще научите повече за логистичната регресия по-късно. Помислете за някои въпроси, които можете да зададете на данните, и кой от тези методи би бил по-подходящ.

В тази секция ще работите с [малък набор от данни за диабет](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Представете си, че искате да тествате лечение за пациенти с диабет. Моделите за машинно обучение могат да ви помогнат да определите кои пациенти биха реагирали по-добре на лечението, въз основа на комбинации от променливи. Дори един много базов модел за регресия, когато е визуализиран, може да покаже информация за променливи, които биха ви помогнали да организирате теоретичните си клинични изпитания.

И така, нека започнем с тази задача!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>

<!--![Илюстрация от \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Илюстрация от @allison_horst-->


## 1. Зареждане на нашия инструментариум

За тази задача ще ни трябват следните пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) е [колекция от R пакети](https://www.tidyverse.org/packages), създадена да направи анализа на данни по-бърз, лесен и забавен!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) е [рамка от пакети](https://www.tidymodels.org/packages/) за моделиране и машинно обучение.

Можете да ги инсталирате със следната команда:

`install.packages(c("tidyverse", "tidymodels"))`

Скриптът по-долу проверява дали имате необходимите пакети за завършване на този модул и ги инсталира, ако някои липсват.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Сега нека заредим тези страхотни пакети и да ги направим достъпни в текущата ни R сесия. (Това е само за илюстрация, `pacman::p_load()` вече го направи за вас)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Диабетният набор от данни

В това упражнение ще демонстрираме уменията си за регресия, като правим прогнози върху набор от данни за диабет. [Диабетният набор от данни](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) включва `442 проби` с данни за диабет, с 10 променливи за прогнозиране: `възраст`, `пол`, `индекс на телесна маса`, `средно кръвно налягане` и `шест измервания на кръвния серум`, както и променлива за резултат `y`: количествена мярка за прогресията на заболяването една година след началната точка.

|Брой наблюдения|442|
|----------------------|:---|
|Брой предиктори|Първите 10 колони са числови предиктивни|
|Резултат/Цел|Колона 11 е количествена мярка за прогресията на заболяването една година след началната точка|
|Информация за предикторите|- възраст в години
||- пол
||- bmi индекс на телесна маса
||- bp средно кръвно налягане
||- s1 tc, общ серумен холестерол
||- s2 ldl, липопротеини с ниска плътност
||- s3 hdl, липопротеини с висока плътност
||- s4 tch, общ холестерол / HDL
||- s5 ltg, вероятно логаритъм на нивото на серумните триглицериди
||- s6 glu, ниво на кръвна захар|

> 🎓 Запомнете, това е контролирано обучение и ни е необходима целева променлива 'y'.

Преди да можете да манипулирате данни с R, трябва да импортирате данните в паметта на R или да изградите връзка към данните, която R може да използва за достъп до тях дистанционно.

> Пакетът [readr](https://readr.tidyverse.org/), който е част от Tidyverse, предоставя бърз и удобен начин за четене на правоъгълни данни в R.

Сега, нека заредим диабетния набор от данни, предоставен на този URL адрес: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Също така ще направим проверка на данните с помощта на `glimpse()` и ще покажем първите 5 реда с помощта на `slice()`.

Преди да продължим, нека въведем нещо, което често ще срещате в R код 🥁🥁: операторът pipe `%>%`

Операторът pipe (`%>%`) изпълнява операции в логическа последователност, като предава обект напред към функция или израз за извикване. Можете да мислите за оператора pipe като "и след това" във вашия код.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` ни показва, че тези данни съдържат 442 реда и 11 колони, като всички колони са от тип данни `double`.

<br>

> glimpse() и slice() са функции в [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, част от Tidyverse, е граматика за манипулиране на данни, която предоставя последователен набор от действия, помагащи за решаването на най-често срещаните предизвикателства при обработката на данни.

<br>

Сега, след като разполагаме с данните, нека се фокусираме върху една характеристика (`bmi`), която ще използваме за тази задача. Това ще изисква да изберем желаните колони. Как можем да направим това?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) ни позволява да *изберем* (и при нужда да преименуваме) колони в дадена таблица с данни.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Данни за обучение и тестване

Обичайна практика в контролираното обучение е *разделянето* на данните на два подмножества: (обикновено по-голям) набор за обучение на модела и по-малък "задържан" набор, с който да се провери как се е представил моделът.

Сега, когато имаме готови данни, можем да видим дали машината може да помогне да се определи логично разделение между числата в този набор от данни. Можем да използваме пакета [rsample](https://tidymodels.github.io/rsample/), който е част от рамката Tidymodels, за да създадем обект, който съдържа информация за *как* да се разделят данните, а след това още две функции от rsample, за да извлечем създадените набори за обучение и тестване:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Обучение на модел за линейна регресия с Tidymodels

Сега сме готови да обучим нашия модел!

В Tidymodels моделите се задават с помощта на `parsnip()` чрез определяне на три концепции:

-   **Типът** на модела разграничава различни модели като линейна регресия, логистична регресия, модели с дървета за решения и други.

-   **Режимът** на модела включва често срещани опции като регресия и класификация; някои типове модели поддържат и двете, докато други имат само един режим.

-   **Енджинът** на модела е изчислителният инструмент, който ще се използва за настройка на модела. Често това са R пакети, като **`"lm"`** или **`"ranger"`**

Тази информация за модела се съхранява в спецификация на модела, така че нека създадем такава!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

След като моделът бъде *определен*, той може да бъде `оценен` или `обучен`, като се използва функцията [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), обикновено с помощта на формула и някакви данни.

`y ~ .` означава, че ще напаснем `y` като предсказваната стойност/цел, обяснена от всички предиктори/характеристики, т.е. `.` (в този случай имаме само един предиктор: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

От модела можем да видим коефициентите, научени по време на обучението. Те представляват коефициентите на линията на най-добро съответствие, която ни дава най-ниската обща грешка между действителната и прогнозната променлива.
<br>

## 5. Направете прогнози върху тестовия набор

Сега, след като сме обучили модел, можем да го използваме, за да прогнозираме прогресията на заболяването y за тестовия набор от данни, използвайки [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Това ще се използва за начертаване на линията между групите данни.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Ура! 💃🕺 Току-що обучихме модел и го използвахме за правене на прогнози!

При правене на прогнози, конвенцията на tidymodels е винаги да се създава tibble/data frame с резултати и стандартизирани имена на колони. Това улеснява комбинирането на оригиналните данни и прогнозите в удобен формат за последващи операции като визуализация.

`dplyr::bind_cols()` ефективно свързва множество data frame-ове по колони.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Визуализация на резултатите от модела

Сега е време да видим това визуално 📈. Ще създадем разпръснат график на всички стойности на `y` и `bmi` от тестовия набор, след което ще използваме прогнозите, за да начертаем линия на най-подходящото място, между групите данни на модела.

R има няколко системи за създаване на графики, но `ggplot2` е една от най-елегантните и най-гъвкавите. Това ви позволява да съставяте графики чрез **комбиниране на независими компоненти**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Помислете малко какво се случва тук. Една права линия минава през множество малки точки от данни, но какво точно прави тя? Можете ли да видите как бихте могли да използвате тази линия, за да предвидите къде нова, невиждана досега точка от данни би трябвало да се впише спрямо оста y на графиката? Опитайте се да опишете с думи практическото приложение на този модел.

Поздравления, създадохте първия си модел за линейна регресия, направихте предсказание с него и го визуализирахте в графика!



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
